In [ ]:
%pip install yfinance pandas plotly statsmodels

In [ ]:
import yfinance as yf
import pandas as pd

def get_stock_data(symbol: str, start="2020-01-01", end=None):
    """Fetch daily stock data from Yahoo Finance."""
    if end is None:
        end = pd.Timestamp.today().strftime('%Y-%m-%d')
    
    print(f"Fetching {symbol}...")
    df = yf.download(symbol, start=start, end=end, progress=False, auto_adjust=False)
    
    if df.empty:
        print(f"⚠️ Error fetching {symbol}: No data returned")
        return pd.DataFrame()
    
    # Flatten column names if they're multi-level
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    
    print(f"✓ Fetched {len(df)} days of data for {symbol}")
    return df

# Fetch stock data (no throttling needed with yfinance!)
aapl = get_stock_data("AAPL")
nvda = get_stock_data("NVDA")
lyft = get_stock_data("LYFT")

print("\nAAPL Sample:")
print(aapl.head())
print("\nColumns:", aapl.columns.tolist())

In [ ]:
def add_ema_dema(df, span=20):
    """Add EMA and DEMA columns to the DataFrame."""
    df[f"EMA_{span}"] = df["Close"].ewm(span=span, adjust=False).mean()
    ema = df[f"EMA_{span}"]
    df[f"DEMA_{span}"] = 2*ema - ema.ewm(span=span, adjust=False).mean()
    return df

# Apply to all stocks
aapl = add_ema_dema(aapl, 20)
nvda = add_ema_dema(nvda, 20)
lyft = add_ema_dema(lyft, 20)


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_stock(df, symbol, span=20):
    fig = go.Figure()
    
    # Add traces
    fig.add_trace(go.Scatter(x=df.index, y=df["Close"], 
                             name="Close Price", 
                             line=dict(color='blue', width=2)))
    fig.add_trace(go.Scatter(x=df.index, y=df[f"EMA_{span}"], 
                             name=f"EMA {span}", 
                             line=dict(color='orange', dash='dash')))
    fig.add_trace(go.Scatter(x=df.index, y=df[f"DEMA_{span}"], 
                             name=f"DEMA {span}", 
                             line=dict(color='green', dash='dot')))
    
    # Update layout with range slider
    fig.update_layout(
        title=f"{symbol} Stock with EMA & DEMA",
        xaxis_title="Date",
        yaxis_title="Price ($)",
        hovermode='x unified',
        height=600,
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1d", step="day", stepmode="backward"),
                    dict(count=7, label="1w", step="day", stepmode="backward"),
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=3, label="3m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(count=5, label="5y", step="year", stepmode="backward"),
                    dict(step="all", label="All")
                ])
            ),
            rangeslider=dict(visible=True),
            type="date"
        )
    )
    
    fig.show()

plot_stock(aapl, "AAPL")
plot_stock(nvda, "NVDA")
plot_stock(lyft, "LYFT")

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def decompose_series(df, symbol):
    decomposition = seasonal_decompose(df["Adj Close"], model="multiplicative", period=252)
    
    # Create subplots with shared x-axis
    fig = make_subplots(
        rows=4, cols=1,
        subplot_titles=('Observed', 'Trend', 'Seasonal', 'Residual'),
        vertical_spacing=0.08,
        shared_xaxes=True
    )
    
    # Add traces for each component
    fig.add_trace(go.Scatter(x=df.index, y=decomposition.observed, 
                             name='Observed', line=dict(color='blue', width=2)),
                  row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=decomposition.trend, 
                             name='Trend', line=dict(color='orange', width=2)),
                  row=2, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=decomposition.seasonal, 
                             name='Seasonal', line=dict(color='green', width=2)),
                  row=3, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=decomposition.resid, 
                             name='Residual', line=dict(color='red', width=1)),
                  row=4, col=1)
    
    # Update layout
    fig.update_layout(
        title_text=f"{symbol} - Time Series Decomposition",
        height=1000,
        showlegend=False,
        hovermode='x unified',
        template='plotly_white'
    )
    
    # Add y-axis labels
    fig.update_yaxes(title_text="Price ($)", row=1, col=1)
    fig.update_yaxes(title_text="Multiplier", row=2, col=1)
    fig.update_yaxes(title_text="Multiplier", row=3, col=1)
    fig.update_yaxes(title_text="Multiplier", row=4, col=1)
    
    # Add x-axis label to bottom
    fig.update_xaxes(title_text="Date", row=4, col=1)
    
    # Add range selector to top (row 1) and range slider to bottom (row 4)
    fig.update_xaxes(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1d", step="day", stepmode="backward"),
                dict(count=7, label="1w", step="day", stepmode="backward"),
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=3, label="3m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(count=5, label="5y", step="year", stepmode="backward"),
                dict(step="all", label="All")
            ]),
            bgcolor="lightgray",
            activecolor="gray",
            y=1.15,
            yanchor="top"
        ),
        type="date",
        row=1, col=1
    )
    
    fig.update_xaxes(
        rangeslider=dict(visible=True),
        type="date",
        row=4, col=1
    )
    
    fig.show()

decompose_series(aapl, "AAPL")
decompose_series(nvda, "NVDA")
decompose_series(lyft, "LYFT")

In [ ]:
# Step 4: Classical Time Series Models (MA, AR, ARIMA)

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import plotly.graph_objects as go
import numpy as np

# Function to prepare train/test split
def prepare_data(df, train_ratio=0.8):
    data = df["Adj Close"].dropna()
    train_size = int(len(data) * train_ratio)
    train, test = data[:train_size], data[train_size:]
    return train, test

# Prepare data for all stocks
aapl_train, aapl_test = prepare_data(aapl)
nvda_train, nvda_test = prepare_data(nvda)
lyft_train, lyft_test = prepare_data(lyft)

print(f"AAPL - Training: {len(aapl_train)} days, Test: {len(aapl_test)} days")
print(f"NVDA - Training: {len(nvda_train)} days, Test: {len(nvda_test)} days")
print(f"LYFT - Training: {len(lyft_train)} days, Test: {len(lyft_test)} days")

In [ ]:
# Moving Average (MA) Model - MA(q)
# MA models use past forecast errors in a regression-like model

from sklearn.metrics import mean_squared_error, mean_absolute_error

def fit_ma_model(train, test, q=20):
    """Fit MA model and return forecasts and metrics"""
    model = ARIMA(train, order=(0, 0, q))
    fitted = model.fit()
    forecast = fitted.forecast(steps=len(test))
    forecast.index = test.index
    
    rmse = np.sqrt(mean_squared_error(test, forecast))
    mae = mean_absolute_error(test, forecast)
    
    return fitted, forecast, rmse, mae

# Fit MA(20) for all stocks
print("="*60)
print("MOVING AVERAGE (MA) MODEL - MA(20)")
print("="*60)

aapl_ma_fitted, aapl_ma_forecast, aapl_ma_rmse, aapl_ma_mae = fit_ma_model(aapl_train, aapl_test)
print(f"\nAAPL MA(20) - RMSE: ${aapl_ma_rmse:.2f}, MAE: ${aapl_ma_mae:.2f}")

nvda_ma_fitted, nvda_ma_forecast, nvda_ma_rmse, nvda_ma_mae = fit_ma_model(nvda_train, nvda_test)
print(f"NVDA MA(20) - RMSE: ${nvda_ma_rmse:.2f}, MAE: ${nvda_ma_mae:.2f}")

lyft_ma_fitted, lyft_ma_forecast, lyft_ma_rmse, lyft_ma_mae = fit_ma_model(lyft_train, lyft_test)
print(f"LYFT MA(20) - RMSE: ${lyft_ma_rmse:.2f}, MAE: ${lyft_ma_mae:.2f}")

In [ ]:
# Autoregressive (AR) Model - AR(p)
# AR models use past values of the series itself to predict future values

def fit_ar_model(train, test, p=20):
    """Fit AR model and return forecasts and metrics"""
    model = ARIMA(train, order=(p, 0, 0))
    fitted = model.fit()
    forecast = fitted.forecast(steps=len(test))
    forecast.index = test.index
    
    rmse = np.sqrt(mean_squared_error(test, forecast))
    mae = mean_absolute_error(test, forecast)
    
    return fitted, forecast, rmse, mae

# Fit AR(20) for all stocks
print("="*60)
print("AUTOREGRESSIVE (AR) MODEL - AR(20)")
print("="*60)

aapl_ar_fitted, aapl_ar_forecast, aapl_ar_rmse, aapl_ar_mae = fit_ar_model(aapl_train, aapl_test)
print(f"\nAAPL AR(20) - RMSE: ${aapl_ar_rmse:.2f}, MAE: ${aapl_ar_mae:.2f}")

nvda_ar_fitted, nvda_ar_forecast, nvda_ar_rmse, nvda_ar_mae = fit_ar_model(nvda_train, nvda_test)
print(f"NVDA AR(20) - RMSE: ${nvda_ar_rmse:.2f}, MAE: ${nvda_ar_mae:.2f}")

lyft_ar_fitted, lyft_ar_forecast, lyft_ar_rmse, lyft_ar_mae = fit_ar_model(lyft_train, lyft_test)
print(f"LYFT AR(20) - RMSE: ${lyft_ar_rmse:.2f}, MAE: ${lyft_ar_mae:.2f}")


# ARIMA Model - ARIMA(p, d, q)
# Combines AR and MA with differencing (d) for non-stationary data

def fit_arima_model(train, test, p=20, d=1, q=20):
    """Fit ARIMA model and return forecasts and metrics"""
    model = ARIMA(train, order=(p, d, q))
    fitted = model.fit()
    forecast = fitted.forecast(steps=len(test))
    forecast.index = test.index
    
    rmse = np.sqrt(mean_squared_error(test, forecast))
    mae = mean_absolute_error(test, forecast)
    
    return fitted, forecast, rmse, mae

# Fit ARIMA(20,1,20) for all stocks
print("\n" + "="*60)
print("ARIMA MODEL - ARIMA(20,1,20)")
print("="*60)

aapl_arima_fitted, aapl_arima_forecast, aapl_arima_rmse, aapl_arima_mae = fit_arima_model(aapl_train, aapl_test)
print(f"\nAAPL ARIMA(20,1,20) - RMSE: ${aapl_arima_rmse:.2f}, MAE: ${aapl_arima_mae:.2f}")

nvda_arima_fitted, nvda_arima_forecast, nvda_arima_rmse, nvda_arima_mae = fit_arima_model(nvda_train, nvda_test)
print(f"NVDA ARIMA(20,1,20) - RMSE: ${nvda_arima_rmse:.2f}, MAE: ${nvda_arima_mae:.2f}")

lyft_arima_fitted, lyft_arima_forecast, lyft_arima_rmse, lyft_arima_mae = fit_arima_model(lyft_train, lyft_test)
print(f"LYFT ARIMA(20,1,20) - RMSE: ${lyft_arima_rmse:.2f}, MAE: ${lyft_arima_mae:.2f}")


# Compare all models for all stocks
print("\n" + "="*60)
print("MODEL COMPARISON - ALL STOCKS")
print("="*60)

for symbol, ma_rmse, ma_mae, ar_rmse, ar_mae, arima_rmse, arima_mae in [
    ("AAPL", aapl_ma_rmse, aapl_ma_mae, aapl_ar_rmse, aapl_ar_mae, aapl_arima_rmse, aapl_arima_mae),
    ("NVDA", nvda_ma_rmse, nvda_ma_mae, nvda_ar_rmse, nvda_ar_mae, nvda_arima_rmse, nvda_arima_mae),
    ("LYFT", lyft_ma_rmse, lyft_ma_mae, lyft_ar_rmse, lyft_ar_mae, lyft_arima_rmse, lyft_arima_mae)
]:
    print(f"\n{symbol}:")
    print(f"  MA(20)         - RMSE: ${ma_rmse:.2f}, MAE: ${ma_mae:.2f}")
    print(f"  AR(20)         - RMSE: ${ar_rmse:.2f}, MAE: ${ar_mae:.2f}")
    print(f"  ARIMA(20,1,20) - RMSE: ${arima_rmse:.2f}, MAE: ${arima_mae:.2f}")

In [ ]:
# Visualize Model Predictions vs Actual

def plot_forecasts(train, test, ma_forecast, ar_forecast, arima_forecast, symbol):
    fig = go.Figure()
    
    # Training data
    fig.add_trace(go.Scatter(x=train.index, y=train, 
                             name='Training Data', 
                             line=dict(color='blue', width=2)))
    
    # Actual test data
    fig.add_trace(go.Scatter(x=test.index, y=test, 
                             name='Actual Test Data', 
                             line=dict(color='black', width=2)))
    
    # MA forecast
    fig.add_trace(go.Scatter(x=ma_forecast.index, y=ma_forecast, 
                             name='MA(20) Forecast', 
                             line=dict(color='red', dash='dash')))
    
    # AR forecast
    fig.add_trace(go.Scatter(x=ar_forecast.index, y=ar_forecast, 
                             name='AR(20) Forecast', 
                             line=dict(color='orange', dash='dash')))
    
    # ARIMA forecast
    fig.add_trace(go.Scatter(x=arima_forecast.index, y=arima_forecast, 
                             name='ARIMA(20,1,20) Forecast', 
                             line=dict(color='green', dash='dash')))
    
    # Update layout
    fig.update_layout(
        title=f"{symbol} - Model Forecasts Comparison (20 Lags)",
        xaxis_title="Date",
        yaxis_title="Price ($)",
        hovermode='x unified',
        height=600,
        template='plotly_white',
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=3, label="3m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all", label="All")
                ]),
                bgcolor="lightgray",
                activecolor="gray"
            ),
            rangeslider=dict(visible=True),
            type="date"
        )
    )
    
    fig.show()

# Plot for all stocks
plot_forecasts(aapl_train, aapl_test, aapl_ma_forecast, aapl_ar_forecast, aapl_arima_forecast, "AAPL")
plot_forecasts(nvda_train, nvda_test, nvda_ma_forecast, nvda_ar_forecast, nvda_arima_forecast, "NVDA")
plot_forecasts(lyft_train, lyft_test, lyft_ma_forecast, lyft_ar_forecast, lyft_arima_forecast, "LYFT")